![](image.png)


# Edgar Project


> ### BY Team Helvellyn

<div class="alert alert-block alert-success">
<b>Aim:</b> To provide a user with a number of python modules that would allow them to see whether sentiment in a financial report can be correlated with falling or rising returns from a stock
</div>

## Contents

- The code to download bank reports to a readable form from the online resource Edgar

- The module ref_data that contains functions that will allow us to:

> -  Retrive s&p100 tickers

> -  Calculate daily returns on a stock

> - Combine stock data with report data

> - Download words of sentiment from the Loughran McDonald word list

- Code that counts words of sentiment within a banking report

## The edgar_downloader()





<div class="alert alert-block alert-info">
    <b></b>The library <b>Selenium</b> was used to automate webscraping, we needed both the <b>webdriver</b> to run chrome and <b>keys</b>
</div>




In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys








from urllib import request
import requests
from bs4 import BeautifulSoup 
import time


<div class="alert alert-block alert-warning">
<b></b>Other imports were used in the cleaning of the HTML, to access urls whilst webscraping and time was used to insert pauses
</div>

In [91]:
def download_clean_text(ticker, dest_folder):
    dates = []
    row = 1
    try:
        while row<40:
            row = row+1
            p = {'ticker':ticker, 'dest_folder':dest_folder}
            PATH = "C:\Program Files (x86)\chromedriver.exe"
            driver = webdriver.Chrome(PATH)
            driver.get('https://www.sec.gov/edgar/searchedgar/companysearch.html')
            searchinput_path1 = r'//*[@id="company"]'
            search_input1 = driver.find_element_by_xpath(searchinput_path1)
            search_input1.send_keys(ticker, Keys.ENTER)
            time.sleep(1)   # time sleeps to stop errors
            search_bar = r'//*[@id="type"]'
            driver.find_element_by_xpath(search_bar).click()
            searchinput_path2 = r'//*[@id="type"]'
            search_input2 = driver.find_element_by_xpath(searchinput_path2)
            search_input2.send_keys('10-K', Keys.ENTER)
            url1 = driver.current_url
            time.sleep(1)
            Document_button = r'/html/body/div[4]/div[4]/table/tbody/tr[{}]/td[2]/a[1]'.format(row)
            DB = driver.find_element_by_xpath(Document_button)
            DB.send_keys(Keys.ENTER)
            time.sleep(1)
            filing_date = driver.find_element_by_xpath(r'//*[@id="formDiv"]/div[2]/div[1]/div[2]').text  #to get the date
            date = dates.append(filing_date)
            datepath = open(dest_folder+'/Datelist'+ticker+'.txt', "w", encoding = 'utf8') # collects date information
            datepath.write(str(dates))
            datepath.close()
            time.sleep(1)
            try:
                html_link = r'//*[@id="formDiv"]/div/table/tbody/tr[2]/td[3]/a'
                HL = driver.find_element_by_xpath(html_link)
                HL.send_keys(Keys.ENTER)
                time.sleep(1)
                driver.switch_to.window(driver.window_handles[1])
                htm_url = driver.current_url
                resp = request.urlopen(htm_url)
                data = resp.read()
                html_string = data.decode("UTF-8")
                clean_string = BeautifulSoup(html_string, 'lxml').text
                text_file = open(dest_folder+'/'+ticker+'_10_k_'+filing_date+'.txt', "w", encoding = 'utf8')
                text_file.write(html_link)
                text_file.close()
                driver.quit()
            except:
                try:
                    menu_button = r'//*[@id="menu-dropdown-link"]'
                    MB = driver.find_element_by_xpath(menu_button)
                    MB.send_keys(Keys.ENTER)
                    time.sleep(1)
                    open_html = r'//*[@id="form-information-html"]'
                    OH = driver.find_element_by_xpath(open_html)
                    OH.send_keys(Keys.ENTER)
                    time.sleep(1)
                    driver.switch_to.window(driver.window_handles[1])
                    htm_url = driver.current_url
                    resp = request.urlopen(htm_url)
                    data = resp.read()
                    html_string = data.decode("UTF-8")
                    clean_string = BeautifulSoup(html_string, 'lxml').text
                    text_file = open(dest_folder+'/'+ticker+'_10_k_'+filing_date+'.txt', "w", encoding = 'utf8')
                    text_file.write(clean_string)
                    text_file.close()
                    driver.quit()
                except:
                    htm_url = driver.current_url
                    resp = request.urlopen(htm_url)
                    data = resp.read()
                    html_string = data.decode("UTF-8")
                    clean_string = BeautifulSoup(html_string, 'lxml').text
                    text_file = open(dest_folder+'/'+ticker+'_10_k_'+filing_date+'.txt', "w", encoding = 'utf8')
                    text_file.write(clean_string)
                    text_file.close()
            driver.quit()
    except:
        print('Downloads complete!')

<div class="alert alert-block alert-info">
    <b></b>Accessing the website and searching for a ticker
    
</div>

In [10]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get('https://www.sec.gov/edgar/searchedgar/companysearch.html')
time.sleep(4)
searchinput_path1 = r'//*[@id="company"]' # XPath
search_input1 = driver.find_element_by_xpath(searchinput_path1)
search_input1.send_keys('APPL', Keys.ENTER) 
time.sleep(5)
driver.quit()

<div class="alert alert-block alert-info">
    <b> Finding the XPath</b> 
</div>

![](inspect1.png)

![](copy1.png)

<div class="alert alert-block alert-success">
    <b></b>In order to retrive every 10K report we had to iterate through each row, in order to this we defined the variable <b>row</b> </div>

![](docubutton.png)

In [ ]:
row = 1
row < 40:
    row=row+1
    Document_button = r'/html/body/div[4]/div[4]/table/tbody/tr[{}]/td[2]/a[1]'.format(row)

<div class="alert alert-block alert-warning">
    <b>
        Scraping the date</b> - dates were not specified on the file name so before selecting the report we had to get the date
    </div>
   

![](dateh.png)

In [ ]:
# defining date
filing_date = driver.find_element_by_xpath(r'//*[@id="formDiv"]/div[2]/div[1]/div[2]').text  

# defining file name
text_file = open(dest_folder+'/'+ticker+'_10_k_'+filing_date+'.txt', "w", encoding = 'utf8')

<div class="alert alert-block alert-danger">
    <b>Downloading the correct url</b> 
    </div>

In [107]:
driver.switch_to.window(driver.window_handles[1])
htm_url = driver.current_url

![](tab.png)

### Cleaning to a text file

In [ ]:
resp = request.urlopen(htm_url)
data = resp.read()
html_string = data.decode("UTF-8")
clean_string = BeautifulSoup(html_string, 'lxml').text
text_file = open(dest_folder+'/'+ticker+'_10_k_'+filing_date+'.txt', "w", encoding = 'utf8')
text_file.write(clean_string)
text_file.close()

<div class="alert alert-block alert-success">
    <b>lxml</b> parser, this removes all HTML characters leaving a clean readable text file
</div>

![](HTML.png)

![](clean.png)

# Calling the function

In [12]:
import edgar_downloader
edgar_downloader.download_clean_text('AMZN', 'C:/10k_filings_clean')

![](txtfiles3.png)

## ref_data module 

- get_sp100() to return tickers 

- get_yahoo_data()

- get_yahoo_data_visual()

- get_sentiment_word_dict()

#### get_sp100()

<div class= 'alert alert-block alert-info'>
<b></b> Returning all s&p100 tickers from wikipedia, using Beautifulsoup to access the table data
</div>

In [ ]:
def get_sp100():
    import requests
    from bs4 import BeautifulSoup
    url = 'https://en.wikipedia.org/wiki/S&P_100'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table_soup = soup.find_all('table', {'class':'wikitable sortable'})

<div class='alert alert-block alert-warning'
<b></b> 1st loop iterates through the table rows

2nd loop iterates through the table and cleans it
</div>

In [ ]:
Tickers = []
# 1st loop
for i in table_soup[0].find_all('tr')[1:]:
    td = i.find_all('td')
    Tickers.append(td[0].text)
tickers_final = []

# 2nd loop
for i in Tickers:
    i = i.replace('\n','')
    tickers_final.append(i)
return tickers_final

In [3]:
import ref_data
tickers_sp100 = ref_data.get_sp100()
print(tickers_sp100)

### get_yahoo_data

<div class="alert alert-block alert-success">
    <b></b> We used the library <b>yahoo_fin</b> to access stock information, we had to pip install this on the command prompt
        </div>

In [14]:
import yahoo_fin.stock_info as si
from yahoo_fin.stock_info import get_data, tickers_sp500

<div class="alert alert-block alert-info">
    <b></b> To find the percentage change for n number of days from a day we used the pandas <b>.pct_change()</b> attatchment, this meant we could find n number of days return

    
a <b>shift</b> was also used to ensure the data lined up correctly 
        </div>

In [24]:
# body of code

def get_yahoo_data(ticker, start_date, end_date):
    import yahoo_fin.stock_info as si
    from yahoo_fin.stock_info import get_data, tickers_sp500
    df = si.get_data(ticker, start_date = start_date, end_date = end_date)
    df['1daily_return'] = df.adjclose.pct_change().shift(-1)
    df['2daily_return'] = df.adjclose.pct_change(2).shift(-2)
    df['3daily_return'] = df.adjclose.pct_change(3).shift(-3)
    df['5daily_return'] = df.adjclose.pct_change(5).shift(-5)
    df['10daily_return'] = df.adjclose.pct_change(10).shift(-10)
    return df.head(10)

In [26]:
import ref_data
ref_data.get_yahoo_data('','2000/01/01', '2021/01/14')

#### Linking 10K reports to the stock data 

<div class="alert alert-block alert-warning">
    <b></b>
   <b>get_yahoo_data_visual()</b> 
</div>

<div class='alert alert-block alert-danger'>
    <b></b>We used <b>plotly</b> to visualise the changing stock price
    </div>

In [25]:
import plotly.express as px

df = si.get_data('AlL', start_date = '2000/01/01', end_date = '2021/01/14')
fig = px.line(df, x=df.index, y=['adjclose'])
fig

<div class='alert alert-block alert-success'>
    <b></b>Then added a line signalling the date of a report being released
    </div>

In [16]:
fig.update_layout(
        shapes=[
            dict(
              type= 'line',
              yref= 'paper', y0= 0, y1= 90,
              xref= 'x', x0= '2019-01-01' , x1 = '2019-01-01'
            )
        ]
)

<div class='alert alert-block alert-info'>
    <b>Issue: </b>The report dates were different for each company and did not follow a strict pattern.

So we had to generate a list of dates in the <b>edgar_downloader</b> module, this list is downloaded, and cleaned before being sliced through one by one, generating a line for each report.
    </div>

In [ ]:
# edgar_downloader

dates = []
date = dates.append(filing_date)
datepath = open(dest_folder+'/Datelist'+ticker+'.txt', "w", encoding = 'utf8') # collects date information
datepath.write(str(dates))
datepath.close()


# get_yahoo_data_visual

with open(input_folder+'/Datelist'+ticker+'.txt') as f:
    lines = f.readlines()
res = str(lines[0])[1:-1] 
res = res.replace("'","")
datelist = res.split(',')
fig.update_layout(
    shapes=[
        dict(
          type= 'line',
          yref= 'paper', y0= 0, y1= 90,
          xref= 'x', x0= datelist[0], x1 = datelist[0]
        ), ...

### Calling the function:

In [18]:
import ref_data
ref_data.get_yahoo_data_visual('AMZN', '2000/01/01', '2021/01/14', 'C:/10k_filings_clean')

### get_sentiment_word_dict()

<div class='alert alert-block alert-success'>
     <b></b> Returns a dictionary containing Loughran McDonald sentiment words
     </div>

In [15]:
def get_sentiment_word_dict(sentiment):
    import openpyxl
    from pathlib import Path
    xlsx_file = Path('LoughranMcDonald_SentimentWordLists_2018.xlsx')
    file = openpyxl.load_workbook(xlsx_file)
    sheet = file[sentiment]
    item = []
    for row in sheet.iter_rows():
        for cell in row:
            item.append(cell.value)
    return item

<div class = 'alert alert-block alert-info'>
<b></b> the loop reads through all the sheets on the excel file - and returns each word to its specific category list
</div>

![](excel.png)

In [1]:
import ref_data
ref_data.get_sentiment_word_dict('Negative')

### write_document_sentiments()

<div class='alert alert-block alert-success'>
<b></b> This function counts words of sentiment from a 10K report and enters this count into a dataframe
</div>

<div class='alert alert-block alert-info'>
    <b></b>
We create a dictionary, and append this dictionary with compenents of the filename 
    </div>

In [21]:
# creates dictionary
dir_count = {"Symbol":[],
                 "ReportType":[],
                 "FilingDate":[],
                 "Negative":[],
                "Positive":[],
                "Uncertainty":[],
                "Litigious":[],
                "Constraining":[],
                "WeakModal":[],
                "StrongModal":[]};

# breaks up filename into compentent parts
import os
path = r'C:\10kfilings'
pathname = os.listdir(path)
for i in pathname:
    dir_count['Symbol'].append(i[0:i.find('_')])
    y = [i for i, c in enumerate(i) if c == '_']
    dir_count['ReportType'].append(i[y[0] +1 : y[2]])
    dir_count['FilingDate'].append(i[i.find('.')-10 :i.find('.')])

display(pathname)
display(dir_count)

['AMZN_10_k_2020-01-31.txt']

{'Symbol': ['AMZN'],
 'ReportType': ['10_k'],
 'FilingDate': ['2020-01-31'],
 'Negative': [],
 'Positive': [],
 'Uncertainty': [],
 'Litigious': [],
 'Constraining': [],
 'WeakModal': [],
 'StrongModal': []}

<div class='alert alert-block alert-danger'>
    <b></b>
    We had to call the <b>get_sentiment_word_dict()</b> function that we defined earlier, to return words for Negative, Positive etc. sentiment. 

The <b>filewords</b> list returns the list of words from a report specified with the file variable.
    </div>

In [20]:
# calling previously defined function
Neg = get_sentiment_word_dict('Negative') 
Pos = get_sentiment_word_dict('Positive')
Uncert = get_sentiment_word_dict('Uncertainty')
Lit = get_sentiment_word_dict('Litigious')
Const = get_sentiment_word_dict('Constraining')
WMod = get_sentiment_word_dict('WeakModal')
SMod = get_sentiment_word_dict('StrongModal')


# accesses report
file = open(r"C:\10kfilings\AMZN_10_k_2020-01-31.txt", encoding = 'utf-8').read().split()


# returns words from report
filewords = []
for item in file:
    filewords.append(item.upper()) 
input_words = filewords    
word_count = len(input_words)  

display(Neg)

<div class='alert alert-block alert-warning'>
    <b></b>
We counted words from the report that matched those in the Loughran-McDonald categories. 
    
    
We appended the dictionary with the counts defined. Finally we define the dataframe, <b>count_df</b>.
    </div>

In [ ]:
# creates an 'empty' count
pos_count = 0
neg_count = 0
uncert_count = 0
lit_count = 0
con_count = 0
wmod_count = 0
smod_count = 0


# counting matching words, populating the above
for i in range(0,word_count):
    if input_words[i] in Pos:
        pos_count += 1  
    elif input_words[i] in Neg:
        neg_count += 1
    elif input_words[i] in Uncert:
        uncert_count += 1
    elif input_words[i] in Lit:
        lit_count += 1
    elif input_words[i] in Const:
        con_count += 1
    elif input_words[i] in WMod:
        wmod_count += 1
    elif input_words[i] in SMod:
        smod_count += 1
    else:
        pass


# updating dictionary with counts
dir_count['Positive'] = pos_count
dir_count['Negative'] = neg_count
dir_count['Uncertainty'] = uncert_count
dir_count['Litigious'] = lit_count 
dir_count['Constraining'] = con_count
dir_count['WeakModal'] = wmod_count
dir_count['StrongModal'] = smod_count


# defines dataframe
count_df=pd.DataFrame.from_dict(dir_count)


### Calling the function

In [2]:
import Sentiment
Sentiment.write_document_sentiments(r"C:\10kfilings\AMZN_10_k_2020-01-31.txt")

<div class='alert alert-block alert-warning'>
<b> Limitations and Improvements: </b> 
    
- Future proof it, to allow other directories to be specified
    
- Adding the idea of Negation
    
- Expanding the web scraping to reliable news sources    
    
</div>

## In summary

##### Reviewing our Aim

<div class="alert alert-block alert-success">
<b>Aim:</b> To provide a user with a number of python modules that would allow them to see whether positive or negative sentiment in a financial report can be correlated with falling or rising returns from a stock
</div>

### Any Questions?

In [2]:
# config cell

from traitlets.config.manager import BaseJSONConfigManager
from pathlib import Path
path = Path.home()/".jupyter" / "nbconfig"
cm = BaseJSONConfigManager(config_dir=str(path))
tmp = cm.update(
        "rise",
        {
            "theme":"serif", #blood, league, night,simple, moon, sky, night, solarized, beige, serif
            "transistion":"fade",
            "width":"100%",
            "height":"100%",
            "header":"Kubrick | Team Helvellyn",
            "footer":"EDGAR PROJECT",
            "scroll": True,
            "enable_chalkboard": True,
            "slideNumber": True,
            "center":False,
            "controlsLayout": "edges",
            "hash": True,  
            "backimage": False,
            "autolaunch": True
        }
)